In [8]:
import random

# Parameters
student_groups = ['IF-47-01', 'IF-47-02', 'IF-47-03', 'IF-47-04', 'IF-47-05', 'IF-47-06']
permanent_lectures = ['DOK', 'FAZ', 'GMO', 'IDV', 'SBG', 'PHG', 'ADE', 'AZN', 'DDR', 'DNS', 'DQU', 'DRI', 'DTO']
non_permanent_lectures = ['ALH', 'ALZ', 'APY', 'ASA', 'BRV', 'BTG', 'CDF', 'DAM', 'DRB', 'DVI', 'DWD', 'EDW']
lecturers = permanent_lectures + non_permanent_lectures
rooms = ['TULT-0601', 'TULT-0602', 'TULT-0603', 'TULT-0701', 'TULT-0702', 'TULT-0703', 'TULT-0706', 'TULT-0707', 'TULT-0709', 'TULT-1507', 'TULT-1509', 'TULT-1510']
timeslots = ['timeslot1', 'timeslot2', 'timeslot3', 'timeslot4', 'timeslot5', 'timeslot6', 'timeslot7', 'timeslot8', 'timeslot9', 'timeslot10', 'timeslot11']
courses = ['UKJXB2', 'UAJXA2', 'CII1E3', 'CII1A3', 'CII1C2', 'CII1B3', 'CII1D3', 'CTJ1A2', 'CRI1A2', 'CRI1B3', 'CSI1A2']
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Hyperparameters
num_ants = 10
num_iterations = 100
decay = 0.5
alpha = 1
beta = 2

# Initial pheromone levels
pheromone = [[[1 for _ in timeslots] for _ in rooms] for _ in days]
pheromone_lecturer = {lecturer: 1 for lecturer in lecturers}

# Define the fitness function
def fitness(schedule):
    penalty = 0
    room_timeslot_usage = {}
    lecturer_course_group = {}
    timeslot_count = {room: {day: 0 for day in days} for room in rooms}
    
    for group in schedule:
        for (course, day, room, timeslot, lecturer) in schedule[group]:
            # Check room usage
            if (day, room, timeslot) in room_timeslot_usage:
                penalty += 1
            else:
                room_timeslot_usage[(day, room, timeslot)] = group
                timeslot_count[room][day] += 1

            # Check lecturer constraints
            if (lecturer, course, group) in lecturer_course_group:
                penalty += 1
            else:
                lecturer_course_group[(lecturer, course, group)] = True

    # Penalty for empty timeslots and unbalanced room usage
    for room in rooms:
        for day in days:
            penalty += (len(timeslots) - timeslot_count[room][day]) * 0.1  # Less penalty for fewer empty slots

    return penalty

# Create the initial population
def create_schedule():
    schedule = {}
    for group in student_groups:
        schedule[group] = []
        for course in courses:
            day = random.choice(days)
            room = random.choice(rooms)
            timeslot = random.choice(timeslots)
            lecturer = random.choice(lecturers)
            schedule[group].append((course, day, room, timeslot, lecturer))
    return schedule

# Update pheromones
def update_pheromones(pheromone, pheromone_lecturer, all_schedules):
    for day in range(len(days)):
        for room in range(len(rooms)):
            for timeslot in range(len(timeslots)):
                pheromone[day][room][timeslot] *= decay
                for schedule in all_schedules:
                    if is_course_scheduled(schedule, day, room, timeslot):
                        pheromone[day][room][timeslot] += 1 / (1 + fitness(schedule))
    
    for lecturer in lecturers:
        pheromone_lecturer[lecturer] *= decay
        for schedule in all_schedules:
            if is_lecturer_scheduled(schedule, lecturer):
                pheromone_lecturer[lecturer] += 1 / (1 + fitness(schedule))

    return pheromone, pheromone_lecturer

# Check if a course is scheduled
def is_course_scheduled(schedule, day, room, timeslot):
    for group in schedule:
        for (course, d, r, t, lecturer) in schedule[group]:
            if d == days[day] and r == rooms[room] and t == timeslots[timeslot]:
                return True
    return False

# Check if a lecturer is scheduled
def is_lecturer_scheduled(schedule, lecturer):
    for group in schedule:
        for (course, day, room, timeslot, l) in schedule[group]:
            if l == lecturer:
                return True
    return False

# Main ACO loop
def aco():
    best_schedule = None
    best_fitness = float('inf')
    for iteration in range(num_iterations):
        all_schedules = []
        for ant in range(num_ants):
            schedule = create_schedule()
            all_schedules.append(schedule)
            fit = fitness(schedule)
            if fit < best_fitness:
                best_fitness = fit
                best_schedule = schedule
        update_pheromones(pheromone, pheromone_lecturer, all_schedules)
    return best_schedule, best_fitness

# Display the best schedule in a timetable form
def print_timetable_by_room(schedule):
    timetable = {room: {day: {timeslot: None for timeslot in timeslots} for day in days} for room in rooms}
    for group in schedule:
        for (course, day, room, timeslot, lecturer) in schedule[group]:
            timetable[room][day][timeslot] = (course, group, lecturer)

    for room in rooms:
        print(f"Timetable for {room}:")
        for day in days:
            print(f"  {day}:")
            for timeslot in timeslots:
                entry = timetable[room][day][timeslot]
                if entry:
                    course, group, lecturer = entry
                    print(f"    {timeslot}: {course} for {group} with {lecturer}")
                else:
                    print(f"    {timeslot}: Free")
        print()

best_schedule, best_fitness = aco()
print_timetable_by_room(best_schedule)
print("Best Fitness:", best_fitness)

Timetable for TULT-0601:
  Monday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free
    timeslot11: Free
  Tuesday:
    timeslot1: CII1B3 for IF-47-03 with SBG
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: CII1C2 for IF-47-02 with EDW
    timeslot9: Free
    timeslot10: Free
    timeslot11: Free
  Wednesday:
    timeslot1: Free
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslot10: Free
    timeslot11: Free
  Thursday:
    timeslot1: CII1C2 for IF-47-03 with DQU
    timeslot2: Free
    timeslot3: Free
    timeslot4: Free
    timeslot5: Free
    timeslot6: Free
    timeslot7: Free
    timeslot8: Free
    timeslot9: Free
    timeslo